## Expanding the knowledge

Let's say that now we want to expand our existing knowledge graph with
additional information to enrich the dataset, provide more context and retrieve
more relevant data. 

In this example, we will take **unstructured data**, such as the
character description summary provided below, extract entities from that
summary, generate triplets to build the knowledge graph create queries and
eventually execute those queries in Memgraph to incorporate with the existing
graph. 


This highlights the possibility of loading an unstructured data into the Memgraph. 

Here is an example of unstructured data: 

In [ ]:
# Sample text summary for processing
summary = """
    Viserys Targaryen is the last living son of the former king, Aerys II Targaryen (the 'Mad King').
    As one of the last known Targaryen heirs, Viserys Targaryen is obsessed with reclaiming the Iron Throne and 
    restoring his family’s rule over Westeros. Ambitious and arrogant, he often treats his younger sister, Daenerys Targaryen, 
    as a pawn, seeing her only as a means to gain power. His ruthless ambition leads him to make a marriage alliance with 
    Khal Drogo, a powerful Dothraki warlord, hoping Khal Drogo will give him the army he needs. 
    However, Viserys Targaryen’s impatience and disrespect toward the Dothraki culture lead to his downfall;
    he is ultimately killed by Khal Drogo in a brutal display of 'a crown for a king' – having molten gold poured over his head. 
    """

### Entity extraction

The first step in the process is to extract entities from the summary using
[SpaCy’s LLM](https://spacy.io/usage/large-language-models).

To begin, we need to install SpaCy and the specific model we wll be using.

In [ ]:
%pip install spacy
%pip install spacy_llm
!python -m spacy download en_core_web_md

We are extracting entities from the text, that is, preprocessing the data before
sending it to the GPT model, to get more accurate and relevant results. By
using SpaCy, we can identify key entities such as characters and locations
for a better understanding of the semantics in the text.

This is useful because SpaCy is specifically trained to recognize
linguistic patterns and relationships in text, which helps to isolate and
highlight the most important pieces of information. By preprocessing the text
this way, we ensure that the GPT model receives a more structured input, helps
reduce noise and irrelevant data, leading to more precise and context-aware
outputs. 

In [ ]:
import os
import spacy
from spacy_llm.util import assemble
import json
from collections import Counter
from pathlib import Path

# Split document into sentences
def split_document_sent(text, nlp):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]


def process_text(text, nlp, verbose=False):
    doc = nlp(text)
    if verbose:
        print(f"Text: {doc.text}")
        print(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")
    return doc


# Pipeline to run entity extraction
def extract_entities(text, nlp, verbose=False):
    processed_data = []
    entity_counts = Counter()

    sentences = split_document_sent(text, nlp)
    for sent in sentences:
        doc = process_text(sent, nlp, verbose)
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        # Store processed data for each sentence
        processed_data.append({"text": doc.text, "entities": entities})

        # Update counters
        entity_counts.update([ent[1] for ent in entities])

    # Export to JSON
    with open("processed_data.json", "w") as f:
        json.dump(processed_data, f)



### Generate queries

After the spacyLLM has pre-processed the entities, the data is passed to the GPT model to generate structured data consisting of nodes and relationships. From that, we generate the Cypher queries which will be executed in Memgraph.

In [ ]:
def generate_cypher_queries(nodes, relationships):
    queries = []

    # Create nodes
    for node in nodes:
        query = f"""
        MERGE (n:{node['type']}:Entity {{name: '{node['name']}'}}) 
        ON CREATE SET n.id={node['id']} 
        ON MATCH SET n.id={node['id']}
        """
        queries.append(query)

    # Create relationships
    for rel in relationships:
        query = f"MATCH (a {{id: {rel['source']}}}), (b {{id: {rel['target']}}}) " \
                f"CREATE (a)-[:{rel['relationship']}]->(b)"
        queries.append(query)

    return queries

### Enriching the graph

The `enrich_graph_data` function will merge new knowledge into the graph by doing the following:

1. Extracting the entities with SpacyLLM into JSON
2. Creating nodes and relationships based on extracted entities with GPT model
3. Loading data into Memgraph

In [ ]:
    
def enrich_graph_data(driver, summary):
    nest_asyncio.apply()
    
    load_dotenv()
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    client = AsyncOpenAI()

    # Load the spaCy model
    nlp = spacy.load("en_core_web_md")

    # Sample text summary for processing
    summary = """
        Viserys Targaryen is the last living son of the former king, Aerys II Targaryen (the 'Mad King').
        As one of the last known Targaryen heirs, Viserys Targaryen is obsessed with reclaiming the Iron Throne and 
        restoring his family’s rule over Westeros. Ambitious and arrogant, he often treats his younger sister, Daenerys Targaryen, 
        as a pawn, seeing her only as a means to gain power. His ruthless ambition leads him to make a marriage alliance with 
        Khal Drogo, a powerful Dothraki warlord, hoping Khal Drogo will give him the army he needs. 
        However, Viserys Targaryen’s impatience and disrespect toward the Dothraki culture lead to his downfall;
        he is ultimately killed by Khal Drogo in a brutal display of 'a crown for a king' – having molten gold poured over his head. 
    """

    extract_entities(summary, nlp)

    # Load processed data from JSON
    json_path = Path("processed_data.json")
    with open(json_path, "r") as f:
        processed_data = json.load(f)

    # Prepare nodes and relationships
    nodes = []
    relationships = []

    # Formulate a prompt for GPT-4
    prompt = (
        "Extract entities and relationships from the following JSON data. For each entry in data['entities'], "
        "create a 'node' dictionary with fields 'id' (unique identifier), 'name' (entity text), and 'type' (entity label). "
        "For entities that have meaningful connections, define 'relationships' as dictionaries with 'source' (source node id), "
        "'target' (target node id), and 'relationship' (type of connection). Create max 30 nodes, format relationships in the format of capital letters and _ inbetween words and format the entire response in the JSON output containing only variables nodes and relationships without any text inbetween. Use following labels for nodes: Character, Title, Location, House, Death, Event, Allegiance and following relationship types: HAPPENED_IN, SIBLING_OF, PARENT_OF, MARRIED_TO, HEALED_BY, RULES, KILLED, LOYAL_TO, BETRAYED_BY. Make sure the entire JSON file fits in the output"
        "JSON data:\n"
        f"{json.dumps(processed_data)}"
    )

    response = asyncio.run(get_response(client, prompt))

    structured_data = json.loads(response)  # Assuming GPT-4 outputs structured JSON

    # Populate nodes and relationships lists
    nodes.extend(structured_data.get("nodes", []))
    relationships.extend(structured_data.get("relationships", []))

    cypher_queries = generate_cypher_queries(nodes, relationships)
    with driver.session() as session:
        for query in cypher_queries:
            try:
                session.run(query)
                print(f"Executed query: {query}")
            except Exception as e:
                print(f"Error executing query: {query}. Error: {e}")


enrich_graph_data(driver, summary)

The knowledge graph now has additional knowledge, that is being enriched from unstructured text. 